## Importing some stuff

In [ ]:
import simpy
import random
import gym
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
def time_convert(time):
    time_max = 200
    time_min = -200 
    if time_min < time <= time_max:
        min_max_time = (time-time_min)/(time_max-time_min)
        scaled_time = min_max_time * 20 # Scaling time to be between 0 and 20
        return int(np.ceil(scaled_time))
    else:
        raise Exception("Input time value outside range in def time_convert")
    
    

In [ ]:
time_convert(-199)

- f(`machine.queue`) => OpenAI State

In [ ]:
import gym 

import itertools 

import matplotlib 

import matplotlib.style 

import numpy as np 

import pandas as pd 

import sys 

from collections import defaultdict 
import plotting 
import numpy as np
  

matplotlib.style.use('ggplot') 

### Flow shop environment

In [ ]:
observation_space = gym.spaces.MultiDiscrete((3,3,3,3,21,21,21,21,3))

In [ ]:
observation_space.sample()

In [ ]:
observation_space = gym.spaces.Tuple((gym.spaces.Discrete(3),gym.spaces.Discrete(3),
                                                  gym.spaces.Discrete(3),gym.spaces.Discrete(3),
                                                  gym.spaces.Discrete(21),gym.spaces.Discrete(21),
                                                  gym.spaces.Discrete(21),gym.spaces.Discrete(21),
                                                  gym.spaces.Discrete(3)))

In [ ]:
observation_space.sample()

In [ ]:
class flowshopGym:
    import random
    # --------------------------------------INITIALIZE THINGS------------------------------------------
    def __init__(self):
        self.NUM_PRODUCTS = 20
        self.IAT = 0.001 # change this to sample from some distribution
        self.action_space = gym.spaces.Discrete(2)
        self.DEBUG = 1 # 1 = check simpy env ; 2 = check the openai processes
        #self.observation_space = gym.spaces.Discrete(9)
        # Rich observation space
        # Product type for the 4 product type, time left for the 4 products, product type in machine
        # For more details Thesis notes book on Notability
        self.observation_space = gym.spaces.Tuple((gym.spaces.Discrete(3), # MC1 prod type
                                                  gym.spaces.Discrete(3),gym.spaces.Discrete(3), # MC1 Queue prod type
                                                  gym.spaces.Discrete(3),gym.spaces.Discrete(3),
                                                  gym.spaces.Discrete(11),gym.spaces.Discrete(11), # MC1 Queue time left 
                                                  gym.spaces.Discrete(11),gym.spaces.Discrete(11)
                                                  ))
        #self.observation_space = spaces.Box(-high, high, dtype=np.float32)
        self.prod_and_requests = []
        self.reward = 0 
        self.cumulative_reward = 0 
        self.machine1_prev_type = 0 # The previous product type in the machine
        self.QUEUE_LENGTH = 4
        
        # The following two values from the setup time matrix
        self.P1_to_P2 = 1
        self.P2_to_P1 = 2 
        
        # The range for the time converter
        self.TIME_MAX = 200
        self.TIME_MIN = -200
        
        self.state_format = 'Hosp'
    # --------------------------------------DEFINE PROCESSES------------------------------------------
    
    # Process 1 - Spawn of products
    def process_1(self):
        print("Entered new product process")
        
        # Produce X number of products
        for i in range(self.NUM_PRODUCTS):
            
            # Creating instances of the product 
            ###print("creating instance", i)
            self.instances.append(self.product(self,self.envSimpy,'Product_%d' % i, self.store, self.machine))
            
            # IAT TIMEOUT BEFORE THE NEXT SPAWN
            #next_admission = random.expovariate(1 / interarrival_time)
            yield self.envSimpy.timeout(self.IAT)
        
        # After all the products are put into the store, start process 2
        # Process 2 - Getting products from the store and create machine request
        self.envSimpy.process(self.process_2())
    
    class product(object):
        def __init__(self, flowshopGym, envSimpy, name, number, machine):
            
            # Creating the required resources and environment
            self.envSimpy = envSimpy

            # State space variables
            self.prod_type = random.randint(1,2) # the product type of the product.
            
            
            
            # Rest of the variables
            self.production_time = 2
            self.production_end = random.randint(3,80)
            self.name = name
            self.number = number
            self.machine = machine 
            
            ###print(self.name,"created")
            
            # putting the product into the store
            self.envSimpy.process(self.put_store())
            
 

        def put_store(self): # I am putting 
            yield sim.envSimpy.timeout(1)
            ###print("Putting the", self.name, "in the store at", self.envSimpy.now)
            yield sim.store.put(self)
            ###print("Items in the store",len(sim.store.items)," - time -",self.envSimpy.now)
            
        
    
    
    # -----------------------------PROCESS 2-------------------------------------------------  
    def process_2(self): # If machine.queue is less than 4, get a prod from store and create request
        while True:
            if len(self.machine.queue) < self.QUEUE_LENGTH:
                
                # Get product from store
                prod = yield self.store.get()
                
                # Request machine
                ###print("requesting machine for", prod.name)
                req = prod.machine.request()

                self.envSimpy.process(self.process_3(prod, req))
            else:
                yield self.envSimpy.timeout(1)

    # -----------------------------PROCESS 3-------------------------------------------------  
    def process_3(self,prod, req): # Wait for req to go through and then produce and release the machine  
        sim.prod_and_requests.append([prod,req])
        # Wait for the request to succeed
        yield req
        
        
        #Setup process if required
        #print("setup time ### ",self.machine1_prev_type , prod.prod_type, "at", self.envSimpy.now )
        if self.machine1_prev_type == 0:
            ###print("Previous machine type was 0")
            pass # dont' do anything during the first pass
        elif self.machine1_prev_type == prod.prod_type:
            ###print("Same product type between MC and incoming product")
            pass # Dont' do anything if the product types are the same 
        elif prod.prod_type == 2: # current product type = 2 meaning we have to do a changeover from 1 to 2
            yield self.envSimpy.timeout(self.P1_to_P2)
            ###print("Setup time prod1 to prod2 finished at",self.envSimpy.now )
        elif prod.prod_type == 1: # current product type = 1 meaning we have to do a changeover from 2 to 1
            yield self.envSimpy.timeout(self.P2_to_P1)
            ###print("Setup time prod2 to prod1 finished at",self.envSimpy.now )
        else:
            raise Exception("Sorry product type doesn't match available values [for Setup process]")
        #print("Time after setup time", self.envSimpy.now)
        
        # ----------------------------
        ###print('Start production of ' + str(prod.name) + ' at '+ str(self.envSimpy.now)  )

        ###print("Production time", prod.production_time)
        yield self.envSimpy.timeout(prod.production_time)
        ###print('Production finished at' + str(self.envSimpy.now))
        # Changing the previous prod_type on the machine
        self.machine1_prev_type = prod.prod_type
        
        prod.tardiness = self.envSimpy.now - prod.production_end   # -ve is good; +ve is bad 
        
        self.info['Tardiness'] = prod.tardiness
        # End of production 
        #print('End production of ' + str(prod.name) + ' at ' + str(self.envSimpy.now))


        # Wait for an acceptance only when there are products in the machine.queue
        if len(sim.machine.queue) != 0:
            # 1. After production pass the time so that the agent can take action based on the latest state
            # 2. Pass time when the action chosen by the agent is rejection
            i = 0
            while True:
                if i == 0:
                    yield self.envSimpy.timeout(1)
                    print("Timeout after production")
                i += 1

                
                ###print("Initial queue", sim.machine.queue)
                ###print("Initial queue LENGTH", len(sim.machine.queue))
                ###print("Current_action", sim.current_action)

                if sim.current_action == 1: #rejected

                    # rearrange the machine queue to reflect the rejection
                    sim.machine.queue.insert(len(sim.machine.queue), sim.machine.queue.pop(0))
                    ###print("Resultant queue",sim.machine.queue)
                    ###print("\n")
                    #print("Action rejected; Passing time by 1 ", i)
                    #print("Machine queue,prod types ", sim.machine.queue, sim.getObs())
                    #print("Current time " + str(self.envSimpy.now))
                    #print("Machine product type ",)
                    #print("Queue item 1 product type ", )
                    yield self.envSimpy.timeout(1)




                    # rearrange the machine queue to reflect the rejection
                    #self.machine.queue.insert(len(self.machine.queue), self.machine.queue.pop(0))


                    # print the rearranged machine queue
                    #print('#observation_after')
                    #print(envGym.getObservation())



                elif sim.current_action == 0: #if action accepted
                    ###print("Resultant queue",sim.machine.queue)
                    ###print('\n')
                    break
        else:
            pass


        # Decide the reward 
        if prod.tardiness <= 0:
            sim.reward = 10 
        else: 
            sim.reward = 0

        #print("Tardiness is ",prod.tardiness)
        # Production has ended now I have to take the next action using take_action()
        # envGym.machine.queue = envGym.take_action()
        #envGym.machine.queue = []

        # Release the machine
        #yield prod.machine.release(req)
        
        # Release the machine
        yield prod.machine.release(req)

        # timeout before the next check
        #yield env.timeout(1)
    # ----------------------------------------------------------------------------------------------
    # --------------------------------------DEFINE FUNCTIONS------------------------------------------
    # ----------------------------------------------------------------------------------------------
    
    def getObs(self): 
        # CREATING TUPLE/DICT TO HOLD INFO        
        test_state = dict()
        
        # Machine 1 product type - Initial value
        test_state['MC1_prod_type'] = 0
        # Machine 1 queue product type - Initial values
        test_state['MC1_queue1_type'] = 0
        test_state['MC1_queue2_type'] = 0
        test_state['MC1_queue3_type'] = 0
        test_state['MC1_queue4_type'] = 0 
        # Machine 1 queue time left - Initial values
        test_state['MC1_queue1_timeleft'] = 0
        test_state['MC1_queue2_timeleft'] = 0
        test_state['MC1_queue3_timeleft'] = 0
        test_state['MC1_queue4_timeleft'] = 0
        
        # 1. PRODUCT TYPE INSIDE THE MACHINE
        if len(sim.machine.users) != 0: #if the machine is not empty
            test_state['MC1_prod_type'] = (self.relater(sim.machine.users[0])).prod_type # Product type of the product inside machine 1
        else:
            pass # No product inside the machine
        
        
        # 2. PRODUCT TYPE IN MACHINE 1 QUEUE
        if len(sim.machine.queue) == 0: # queue is empty
            pass # Already defined these values while creating the dict()

        elif len(sim.machine.queue) == 1:
            # Machine 1 queue1 product type
            test_state['MC1_queue1_type'] = (self.relater(sim.machine.queue[0])).prod_type
            
        elif len(sim.machine.queue) == 2:
            # Machine 1 queue1 product type
            test_state['MC1_queue1_type'] = (self.relater(sim.machine.queue[0])).prod_type
            
            # Machine 1 queue2 product type
            test_state['MC1_queue2_type'] = (self.relater(sim.machine.queue[1])).prod_type
        
        elif len(sim.machine.queue) == 3:
            # Machine 1 queue1 product type
            test_state['MC1_queue1_type'] = (self.relater(sim.machine.queue[0])).prod_type
            
            # Machine 1 queue2 product type
            test_state['MC1_queue2_type'] = (self.relater(sim.machine.queue[1])).prod_type
            
            # Machine 1 queue3 product type
            test_state['MC1_queue3_type'] = (self.relater(sim.machine.queue[2])).prod_type
        
        elif len(sim.machine.queue) == 4:
            # Machine 1 queue1 product type
            test_state['MC1_queue1_type'] = (self.relater(sim.machine.queue[0])).prod_type
            
            # Machine 1 queue2 product type
            test_state['MC1_queue2_type'] = (self.relater(sim.machine.queue[1])).prod_type
            
            # Machine 1 queue3 product type
            test_state['MC1_queue3_type'] = (self.relater(sim.machine.queue[2])).prod_type
            
            # Machine 1 queue4 product type
            test_state['MC1_queue4_type'] = (self.relater(sim.machine.queue[3])).prod_type
        
        else:
            raise Exception("Queue length incompatible")

        
        # 3. TIME LEFT CALCULATION
        # for each position
        if test_state['MC1_queue1_type'] != 0: # product present
            time_left = self.time_left((self.relater(sim.machine.queue[0])).production_end)
            test_state['MC1_queue1_timeleft'] = self.time_convert(time_left)
                
        if test_state['MC1_queue2_type'] != 0: # product present
            time_left = self.time_left((self.relater(sim.machine.queue[1])).production_end)
            test_state['MC1_queue2_timeleft'] = self.time_convert(time_left)
        
        if test_state['MC1_queue3_type'] != 0: # product present
            time_left = self.time_left((self.relater(sim.machine.queue[2])).production_end)
            test_state['MC1_queue3_timeleft'] = self.time_convert(time_left)
        
        if test_state['MC1_queue4_type'] != 0: # product present
            time_left = self.time_left((self.relater(sim.machine.queue[3])).production_end)
            test_state['MC1_queue4_timeleft'] = self.time_convert(time_left)
            
        obs = [v for k,v in test_state.items()]    
        #print("------ obs ", tuple(obs))
        
        return tuple(obs)
    
    def time_left(self, due):
        return (due - sim.envSimpy.now)
    
    def time_convert(self, time):
        time_max = self.TIME_MAX
        time_min = self.TIME_MIN
        
        if time_min < time <= time_max:
            min_max_time = (time-time_min)/(time_max-time_min)
            scaled_time = min_max_time * 10 # Scaling time to be between 0 and 20
            return int(np.ceil(scaled_time))
        else:
            raise Exception("Input time value outside range in def time_convert")
    
     
    def encode(self, obs):
        print('Entered encode')
        # (3), 3 [Machine queue type, machine product type]
        i = obs[0] 
        i *= 3
        
        i += obs[1]
        #print("Value of i inside encode ", i)
        return i
    
    def doneFn(self):
        # So the condition is, 
        #(no items in queue1 + queue2 + ...) AND 
        #(no machine is running)
        if self.machine.queue != None:
            lenQueue = len(self.machine.queue)
        else: 
            lenQueue = 0 
        lenMachinesUsed = self.machine.count
        #print("Queue length",lenQueue, "no machines used", lenMachinesUsed)
        if lenQueue == 0 and lenMachinesUsed == 0: 
            #print("length of queue ",lenQueue,"Machines used ", lenMachinesUsed)
            return True
        else:
            #print("length of queue ",lenQueue,"Machines used ", lenMachinesUsed)
            return False
    
    # A function to relate products and requests
    def relater(self,item):
        # Note that the input to this function shouldn't be a list. It should be of 
        # type request or product
        #print("Entered relater with this item ###", item, type(item))
        output_item = None
        
        if str(type(item)) != "<class '__main__.flowshopGym.product'>" and str(type(item)) != 'simpy.resources.resource.Request':
            raise Exception("Passed in a list to relater. Expecting request or product object")
            #print("Error passed in a list. Expecting request or product object")
            return None
        # If the input is a request
        #print("self.prod_and_requests", self.prod_and_requests)
        if type(item) == simpy.resources.resource.Request:
            for i, j in enumerate(self.prod_and_requests):
                if j[1] == item:
                    output_item = j[0]
                    return output_item
        else: #if the input is a product 
            for i, j in enumerate(self.prod_and_requests):
                if j[0] == item:
                    output_item = j[1]
                    return output_item   
    
    
    # --------------------------------------RENDER------------------------------------------
    
    
    
    
    # --------------------------------------RESET------------------------------------------
    def reset(self):
        print("Entered Reset")
        # Initialise simpy environemnt and a machine
        self.envSimpy = simpy.Environment()
        self.machine = simpy.Resource(self.envSimpy,capacity = 1)
        self.store = simpy.Store(self.envSimpy) 
        
        # Initial variable definitions
        self.time_start = self.envSimpy.now
        self.next_time_stop = self.time_start + 1
        self.time_step = 1
        self.time_step_terminal = self.time_start + 100
        self.current_action = 0
        self.instances = []
         
        # Set up starting processes
        self.envSimpy.process(self.process_1())
        
        # Set starting state values

        
        # Inital load of patients (to average occupancy)
        #self._load_patients()
        
        # Starting values of observations
        observations = self.getObs()
        
        # Put state dictionary items into observations list (Define observations)
        #observations = [v for k,v in self.state.items()]
        return observations
    
    
    
    
    # --------------------------------------STEP------------------------------------------
    
    def step(self, action):
        
        # Define params
        self.current_action = action
        self.action_taken = False 
        self.info = {}
        self.reward = 0 
        #print("machine is used by",sim.machine.users)
        # Run for time_step
        self.next_time_stop += self.time_step
        self.envSimpy.run(until = self.next_time_stop)
    
        # Get values
        observation = self.getObs()
        done = self.doneFn()
        info = self.info
        self.cumulative_reward += self.reward
        #print("Values from Step",observation, sim.reward, done, info)
        # Return values
        return (observation, sim.reward, done, info)

        
    

#### Random agent

In [ ]:
sim = flowshopGym()
state = sim.reset()
epochs = 0
penalties, reward, epoch_reward = 0, 0, 0 
epsilon = 0.1
done = False
tardiness = []

while not done:
    
    # Random action
    action = sim.action_space.sample()
    
    # FIFO
    #action = 0
    
    
    #action = np.argmax(Q1[state])
    #if random.uniform(0, 1) < epsilon:
    #    action = sim.action_space.sample() # Explore action space
    #else:
    #    action = np.argmax(Q1[state]) # Exploit learned values
    ###print("###Taking an action at", sim.envSimpy.now)
    state, reward, done, info = sim.step(action)
    if reward == 0:
        penalties += 1
    try:
        tardiness.append(info['Tardiness'])
    except:
        pass
    
    epoch_reward += reward
    

    epochs += 1

    
mean_tardiness = sum(tardiness)/(len(tardiness))
print("End time is ", sim.envSimpy.now)
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))
print("Epoch reward ", epoch_reward)
print("Mean tardiness of epoch ", mean_tardiness, "(-ve is good)")
#print("Median")

In [ ]:
# Tardiness +ve is bad and -ve is good
plt.plot(tardiness)

In [ ]:
tardiness

In [ ]:
np.mean(tardiness)

### Overall flow - Mine

In [ ]:
sim = flowshopGym()
sim.reset()
while True: # This while is for one full cycle of products (20 nos, currently)

    # Choosing the action
    #action = envGym.action_space.sample() # Random agent
    # action = 0 #FIFO
    #action = random.randint(0,1)
    action = sim.action_space.sample()
    print('Action space from the gym ', action)
    # Taking a step
    print("---------------New Cycle----------------")
    #print("Action from the agent " + str(action))
    obs, reward, done, info = sim.step(action) # the env here is gym
    if done:
        print( "End of simulation at " ,sim.envSimpy.now)
        print("Total cumulative reward ", sim.cumulative_reward)
        break
    print("Reward achieved is ", reward)

    #print("Action chosen by the agent ", action)
    #print("Action taken by the environment ", )
     

In [ ]:
np.floor(-41.72999999999972)

### Overall flow - Q-learning - Taxi env

#### Q-learning


In [ ]:
import numpy as np
#q_table = np.zeros([sim.observation_space.n, sim.action_space.n])
Q1 = defaultdict(lambda: np.zeros(sim.action_space.n)) 

In [ ]:
%%time
sim = flowshopGym()
#sim.reset()

"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []
all_rewards = []
all_tardiness = []

for i in range(1, 101):
    state = sim.reset()

    epochs, penalties, reward, episodic_reward, tardiness = 0, 0, 0, 0, []
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = sim.action_space.sample() # Explore action space
        else:
            action = np.argmax(Q1[state]) # Exploit learned values

        next_state, reward, done, info = sim.step(action) 
        ###print("state, action", state, action)
        old_value = Q1[state][action]
        next_max = np.max(Q1[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        Q1[state][action] = new_value
        
        episodic_reward += reward
        
        if reward == 0:
            penalties += 1
        
        state = next_state
        epochs += 1
        
        try:
            print(info['Tardiness'])
            tardiness.append(info['Tardiness'])
        except:
            pass

    all_epochs.append(i)
    all_penalties.append(penalties)
    all_rewards.append(episodic_reward)
    all_tardiness.append(np.mean(tardiness))
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

In [ ]:
plt.plot(all_epochs,all_rewards)

In [ ]:
plt.plot(all_epochs,all_penalties)

In [ ]:
plt.plot(all_epochs,all_tardiness)

In [ ]:
np.mean(all_tardiness)

In [ ]:
all_penalties,all_rewards

In [ ]:
Q1

In [ ]:
Q1 

#### Q - learning implementation 2 

In [ ]:
def createEpsilonGreedyPolicy(Q, epsilon, num_actions): 

    """ 

    Creates an epsilon-greedy policy based 

    on a given Q-function and epsilon. 

       

    Returns a function that takes the state 
a
    as an input and returns the probabilities 

    for each action in the form of a numpy array  

    of length of the action space(set of possible actions). 

    """

    def policyFunction(state):
        

        Action_probabilities = np.ones(num_actions, 

                dtype = float) * epsilon / num_actions 

                  

        best_action = np.argmax(Q[state]) 

        Action_probabilities[best_action] += (1.0 - epsilon) 

        return Action_probabilities 

   

    return policyFunction 
 

def qLearning(env, num_episodes, discount_factor = 1.0, 

                            alpha = 0.6, epsilon = 0.1): 

    """ 

    Q-Learning algorithm: Off-policy TD control. 

    Finds the optimal greedy policy while improving 

    following an epsilon-greedy policy"""

       

    # Action value function 

    # A nested dictionary that maps 

    # state -> (action -> action-value). 

    Q = defaultdict(lambda: np.zeros(env.action_space.n)) 

   

    # Keeps track of useful statistics 

    stats = plotting.EpisodeStats( 

        episode_lengths = np.zeros(num_episodes), 

        episode_rewards = np.zeros(num_episodes))     

       

    # Create an epsilon greedy policy function 

    # appropriately for environment action space 

    policy = createEpsilonGreedyPolicy(Q, epsilon, env.action_space.n) 

       

    # For every episode 

    for ith_episode in range(num_episodes): 

           

        # Reset the environment and pick the first action 

        state = env.reset() 
        #print("This is my state format ",state)
           

        for t in itertools.count(): 

               

            # get probabilities of all actions from current state 

            action_probabilities = policy(state) 

   

            # choose action according to  

            # the probability distribution 

            action = np.random.choice(np.arange( 

                      len(action_probabilities)), 

                       p = action_probabilities) 

   

            # take action and get reward, transit to next state 

            next_state, reward, done, _ = env.step(action) 
            print("This is the next state from step ",next_state)
               

            # Update statistics 

            stats.episode_rewards[ith_episode] += reward 

            stats.episode_lengths[ith_episode] = t 

               

            # TD Update 

            best_next_action = np.argmax(Q[next_state])     

            td_target = reward + discount_factor * Q[next_state][best_next_action] 

            td_delta = td_target - Q[state][action] 

            Q[state][action] += alpha * td_delta 

   

            # done is True if episode terminated    

            if done: 

                break

                   

            state = next_state 

       

    return Q, stats 

In [ ]:
sim = flowshopGym()

In [ ]:
%%time
Q, stats = qLearning(sim, 1000) 

In [ ]:
plotting.plot_episode_stats(stats)

In [ ]:
Q

In [ ]:
state

In [ ]:
np.argmax(Q[0])

In [ ]:
Q1 = defaultdict(lambda: np.zeros(sim.action_space.n)) 

In [ ]:
for i in Q1:
    print(i)
    break

In [ ]:
np.zeros(sim.action_space.n)

In [ ]:
stats